# Batch Two-Photon Pipeline
Run this notebook inside the `fireantsGH` conda environment on WSL.
It orchestrates two-photon preprocessing and registration for every animal with a YAML metadata file.

In [ ]:
from pathlib import Path

from social_imaging_scripts.metadata.config import StageMode, load_project_config
from social_imaging_scripts.pipeline import iter_animals_with_yaml

cfg = load_project_config()
animals = list(iter_animals_with_yaml())
print(f"Discovered {len(animals)} animals with YAML metadata.")
animal_ids = [animal.animal_id for animal in animals]
animal_ids

In [ ]:
# Pick one or more animals to process.
# By default we grab the first entry so the notebook runs quickly.
#selected_animals = [animal_ids[0]]
selected_animals = [animal_ids[1]]
selected_animals

In [ ]:
# Stage control helper
# Example: cfg_slice.confocal_to_anatomy_registration.mode = StageMode.FORCE

In [ ]:
# COMPARISON MODE: Toggle between original (with crop) vs simplified (no crop)
# 
# Legacy (True): Original working version - hard crop + masks + full prematch seed
# Modern (False): Simplified version - only masks + full prematch seed (no crop)
#
# Both modes seed the FULL prematch (rotation + translation) into FireANTs
USE_LEGACY_ROTATION_CROPPING = False

In [ ]:
from social_imaging_scripts.pipeline import run_pipeline

# Clone the project configuration so tweaks here do not affect other runs.
cfg_slice = cfg.model_copy(deep=True)

# Stage modes: StageMode.SKIP, StageMode.REUSE, StageMode.FORCE
cfg_slice.functional_preprocessing.mode = StageMode.REUSE
cfg_slice.motion_correction.mode = StageMode.REUSE
cfg_slice.anatomy_preprocessing.mode = StageMode.REUSE
cfg_slice.confocal_preprocessing.mode = StageMode.REUSE
cfg_slice.fireants_registration.mode = StageMode.REUSE
cfg_slice.confocal_to_anatomy_registration.mode = StageMode.FORCE
cfg_slice.functional_to_anatomy_registration.mode = StageMode.REUSE

# COMPARISON MODE: Toggle between original vs simplified approach
cfg_slice.confocal_to_anatomy_registration.use_legacy_rotation_cropping = USE_LEGACY_ROTATION_CROPPING

if USE_LEGACY_ROTATION_CROPPING:
    print("📦 LEGACY MODE: Hard crop + masks + full prematch (original working version)")
else:
    print("✨ MODERN MODE: Only masks + full prematch (simplified, no crop)")

result = run_pipeline(
    animal_ids=selected_animals,
    cfg=cfg_slice,
)

print("Pipeline run complete.")
try:
    from IPython.display import display
    display(result.to_dataframe())
except Exception:
    from pprint import pprint
    pprint(list(result.iter_session_records()))
